# Kidney Disease Classification - Colab Training
This notebook is designed to run the training pipeline of the Kidney Disease Classification System on Google Colab using a free GPU.

### Prerequisites
1. Push your local repository to GitHub.
2. Have your `kaggle.json` file ready (from Kaggle Account Settings -> Create New API Token).
3. In Colab, go to **Runtime -> Change runtime type** and select **T4 GPU**.

In [ ]:
# 1. Clone your GitHub Repository
# Replace 'YOUR_USERNAME' and 'YOUR_REPO_NAME' with your actual GitHub details.
# If your repo is private, you will need to use a Personal Access Token (PAT).
!git clone https://github.com/YOUR_USERNAME/YOUR_REPO_NAME.git

# Change directory to the cloned repo
import os
os.chdir('YOUR_REPO_NAME')
print("Current Directory:", os.getcwd())

In [ ]:
# 2. Verify GPU Allocation
!nvidia-smi

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
# 3. Install Required Dependencies
# We install the requirements from the repository.
!pip install -r requirements.txt

# Install Kaggle CLI for downloading the dataset
!pip install kaggle

In [ ]:
# 4. Setup Kaggle API and Load Data
# Upload your kaggle.json file here
from google.colab import files
print("Please upload your kaggle.json file:")
uploaded = files.upload()

# Move kaggle.json to the correct location
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Run the data ingestion pipeline stage
# This will download the 1.6GB dataset and create the k-fold splits
!python src/cnnClassifier/pipeline/stage_01_data_ingestion.py

In [ ]:
# 5. Prepare Base Model
# This stage downloads the VGG16 base model and prepares it for 4-class classification
!python src/cnnClassifier/pipeline/stage_02_prepare_base_model.py

In [ ]:
# 6. Run Training Loop
# This stage will train the model using the GPU.
# Note: This might take a while depending on the number of epochs and folds.
!python src/cnnClassifier/pipeline/stage_03_model_training.py

In [ ]:
# 7. Evaluate Model
# This stage evaluates the trained model and logs metrics.
!python src/cnnClassifier/pipeline/stage_04_model_evaluation.py

In [ ]:
# 8. Save Model Weights and Commit to GitHub
# After training, you can push the updated model and metrics back to your repository.
# Make sure to configure your git credentials.

!git config --global user.email "your-email@example.com"
!git config --global user.name "Your Name"

# Add the trained model and evaluation scores
!git add model/model.h5 scores.json

# Commit the changes
!git commit -m "Add trained model and evaluation scores from Colab"

# Push to GitHub (You will need to use a Personal Access Token for authentication)
# Replace YOUR_TOKEN with your actual GitHub PAT
!git push https://YOUR_TOKEN@github.com/YOUR_USERNAME/YOUR_REPO_NAME.git main